In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [7]:
ed = Website("https://wizearly.ai/")
ed.links

['https://www.linkedin.com/company/wizearly-inc/',
 'https://wizearly.ai',
 'https://wizearly.ai/wizearly-dashboard/',
 'https://wizearly.ai/wizpulse/',
 'https://wizearly.ai/about-us/',
 'https://wizearly.ai/contact-us/',
 'https://wizearly.ai/wizearly-dashboard/',
 'https://wizearly.ai/wizpulse/',
 'https://wizearly.ai/about-us/',
 'https://wizearly.ai/contact-us/',
 'https://mqr.wmb.mybluehost.me/website_9caeabed/contact-us/',
 '#elementor-action%3Aaction%3Dpopup%3Aopen%26settings%3DeyJpZCI6IjcyNiIsInRvZ2dsZSI6ZmFsc2V9',
 'https://mqr.wmb.mybluehost.me/website_9caeabed/contact-us/',
 '#elementor-action%3Aaction%3Dpopup%3Aopen%26settings%3DeyJpZCI6IjcyNiIsInRvZ2dsZSI6ZmFsc2V9',
 '#',
 'https://oaklandnatives.org/attendance-challenges/',
 'https://wizearly.ai/wizpulse/',
 'https://www.linkedin.com/company/wizearly-inc/',
 '#',
 'https://wizearly.ai/privacy-policy/',
 'https://wizearly.ai/account_delete_opt_out/',
 'https://wizearly.ai/wizearly-dashboard/',
 'https://wizearly.ai/wizpul

In [8]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [10]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://wizearly.ai/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://www.linkedin.com/company/wizearly-inc/
https://wizearly.ai
https://wizearly.ai/wizearly-dashboard/
https://wizearly.ai/wizpulse/
https://wizearly.ai/about-us/
https://wizearly.ai/contact-us/
https://wizearly.ai/wizearly-dashboard/
https://wizearly.ai/wizpulse/
https://wizearly.ai/about-us/
https://wizearly.ai/contact-us/
https://mqr.wmb.mybluehost.me/website_9caeabed/contact-us/
#elementor-action%3Aaction%3Dpopup%3Aopen%26settings%3DeyJpZCI6IjcyNiIsInRvZ2dsZSI6ZmFsc2V9
https://mqr.wmb.mybluehost.me/website_9caeabed/contact-us/
#elementor-action%3Aaction%3Dpopup%3Aopen%26settings%3DeyJpZCI6IjcyNiIsInRvZ2dsZSI6ZmFsc2V9
#
https://oaklandnatives.org/attendance-challenges/
https://wizearly.ai/w

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [13]:
get_links("https://wizearly.ai/")

{'links': [{'type': 'about page', 'url': 'https://wizearly.ai/about-us/'},
  {'type': 'contact page', 'url': 'https://wizearly.ai/contact-us/'}]}

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://wizearly.ai/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://wizearly.ai/about-us/'}, {'type': 'contact page', 'url': 'https://wizearly.ai/contact-us/'}, {'type': 'company page', 'url': 'https://wizearly.ai'}]}
Landing page:
Webpage Title:
Homepage - Wizearly
Webpage Contents:
+1(510)224-3935
info@wizearly.com
4096 Piedmont Ave, Suite 268, Oakland, CA 94611
Search
Search
Close this search box.
Solutions
Wizearly Dashboard
Wizpulse
About Us
Contact us
Menu
Solutions
Wizearly Dashboard
Wizpulse
About Us
Contact us
Request Demo
AI-Driven Early Warning Solution for
Chronic Absenteeism
What Wizearly does ?
Predict, Monitor & Prevent Chronic Absenteeism.
Learn more
Data Collection
Multi-source collection of attendance data.
AI Analysis
Data analysis using advanced AI algorithms.
ActionablE Insights
AI Driven dashboard and recommendations.
Parental Engagement
Real time parent engagement using mobile apps.
individual learning path
Individual learning path for each student based on the challe

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("Wizearly AI", "https://wizearly.ai/")

Found links: {'links': [{'type': 'about page', 'url': 'https://wizearly.ai/about-us/'}, {'type': 'contact page', 'url': 'https://wizearly.ai/contact-us/'}]}


"You are looking at a company called: Wizearly AI\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHomepage - Wizearly\nWebpage Contents:\n+1(510)224-3935\ninfo@wizearly.com\n4096 Piedmont Ave, Suite 268, Oakland, CA 94611\nSearch\nSearch\nClose this search box.\nSolutions\nWizearly Dashboard\nWizpulse\nAbout Us\nContact us\nMenu\nSolutions\nWizearly Dashboard\nWizpulse\nAbout Us\nContact us\nRequest Demo\nAI-Driven Early Warning Solution for\nChronic Absenteeism\nWhat Wizearly does ?\nPredict, Monitor & Prevent Chronic Absenteeism.\nLearn more\nData Collection\nMulti-source collection of attendance data.\nAI Analysis\nData analysis using advanced AI algorithms.\nActionablE Insights\nAI Driven dashboard and recommendations.\nParental Engagement\nReal time parent engagement using mobile apps.\nindividual learning path\nIndividual learning path for each student b

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("Wizearly AI", "https://wizearly.ai/")

Found links: {'links': [{'type': 'about page', 'url': 'https://wizearly.ai/about-us/'}, {'type': 'contact page', 'url': 'https://wizearly.ai/contact-us/'}]}


# Wizearly AI Company Brochure

---

**Who We Are:**
Wizearly AI is dedicated to tackling chronic absenteeism in schools through advanced AI-driven solutions. Based in Oakland, California, our mission is to empower schools, parents, and students by providing tools that predict, monitor, and prevent absenteeism challenges.

**What We Do:**
At Wizearly, we recognize the critical impact of absenteeism on a student's learning journey. Our innovative AI system, Wiz-E, analyses historical attendance data to identify at-risk students and recommend tailored intervention strategies. Our solutions include:

- **Wizearly Dashboard:** An interactive platform displaying attendance trends and actionable insights for district administrators.
- **Wizpulse:** Engaging mobile applications for parents that include attendance tracking, progress monitoring, and rewards for positive behavior.
- **Real-Time Monitoring:** Ensuring ongoing tracking of students’ progress as interventions are enacted.

**Our Solutions:**
- **Predictive Absenteeism Tracking**
- **Data Collection & AI Analysis**
- **Parental Engagement Tools**
- **Trends Reporting Dashboard**
- **Gamified Engagement Programs**

**Customer Engagement:**
Our clients range from school districts to individual parents, uniting them in the fight against absenteeism. Wizearly not only empowers educators but also fosters strong partnerships with families, encouraging active participation in their children’s educational journeys.

**Company Culture:**
Wizearly AI values innovation, collaboration, and social impact. Our team comprises passionate individuals dedicated to improving educational outcomes through technology. We believe in inclusivity and transparency, ensuring that every team member contributes to our mission to uplift the educational system.

**Careers at Wizearly:**
Join us in making a difference! We are always on the lookout for enthusiastic individuals who share our vision of combating chronic absenteeism. Exciting career opportunities await those who are eager to work in a dynamic environment that fosters growth and creativity. Visit our website or contact us to inquire about open positions.

**Get in Touch:**
Contact us to learn more about our solutions, request a demo, or inquire about career opportunities.

- **Address:** 4096 Piedmont Ave, Suite 268, Oakland, CA 94611
- **Phone:** +1 (510) 224-3935
- **Email:** info@wizearly.com
- **Working Hours:** Mon - Fri, 8:00 AM - 5:00 PM

**Connect With Us:**
Stay updated and join our community on [LinkedIn](#), [Instagram](#), and [Twitter](#).

---

Together, let's predict, monitor, and prevent chronic absenteeism—ensuring every student has a chance to excel!

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("Wizearly AI", "https://wizearly.ai/")

Found links: {'links': [{'type': 'about page', 'url': 'https://wizearly.ai/about-us/'}, {'type': 'contact page', 'url': 'https://wizearly.ai/contact-us/'}]}


# Wizearly AI Brochure

---

**Contact Information:**
- **Address:**  
  4096 Piedmont Ave, Suite 268,  
  Oakland, CA 94611

- **Phone:**  
  +1 (510) 224-3935

- **Email:**  
  info@wizearly.ai

- **Business Hours:**  
  Mon-Fri: 8:00 AM - 5:00 PM

---

## About Wizearly AI

At Wizearly AI, we focus on a mission to **predict, monitor, and prevent chronic absenteeism** in educational settings. Our innovative solutions leverage advanced artificial intelligence algorithms to provide actionable insights and improve student engagement—and ultimately, academic success.

---

## Our Solutions

### **Wizearly Dashboard**
- **Data Collection:** Multi-source attendance data collection.
- **AI Analysis:** Utilize advanced algorithms to analyze data.
- **Actionable Insights:** Provides insights and recommendations for interventions.
- **Real-Time Monitoring:** Continuous tracking of interventions.

### **Wizpulse**
- **Parental Engagement:** Engage parents through mobile apps available on Android and iOS.
- **Gamified Experience:** Making attendance improvement fun and engaging with rewards and challenges.
- **Customized Learning Paths:** Tailored strategies for each student to combat absenteeism.

---

## Company Culture

Wizearly AI fosters an **inclusive and collaborative culture**, believing that teamwork and diversity drive innovation. We value creativity, accountability, and a shared commitment to empowering educators and families through cutting-edge technology.

---

## Join Our Team

Are you passionate about making an impact in education? At Wizearly AI, we are continually looking for talented individuals to join our team. We inspire our employees to think outside the box and take ownership of projects that contribute to shaping the future of education through innovative AI solutions.

Visit our careers page to explore job opportunities and be a part of our mission to enhance student attendance and engagement!

---

## Our Customers

Our platform is designed for educational institutions looking to effectively address student absenteeism. We empower schools and districts with the tools to identify risks early and facilitate communication between educators and parents to foster better student attendance and performance.

---

## Connect With Us

Follow us on our social media channels to stay updated on our latest news and initiatives:
- [LinkedIn](#)
- [Instagram](#)
- [Twitter](#)

For more information or to request a demo, visit [Wizearly AI's Website](#) or contact us directly at the provided email or phone number.

---

**Wizearly AI - Transforming Attendance Through Innovation**  
© 2025 All Rights Reserved.